In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import numpy as np

In [ ]:
#uploading the CSV file to google collab
from google.colab import files
uploaded = files.upload()

Saving data_1.csv to data_1.csv


In [ ]:
#loading the dataset
data = pd.read_csv(list(uploaded.keys())[0])

In [ ]:
#printing the dataset
print(data.head())

   study  exercise  sleep  exam_score
0      2         1      6          60
1      3         2      7          70
2      5         1      8          85
3      4         3      6          75
4      6         2      7          90


In [ ]:
#seperating X and target variable y
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [ ]:
#min-max scaling on dataset
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#spliting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [ ]:
def backward_elimination(X, y):
    num_vars = len(X[0])
    for i in range(0, num_vars):
        regressor_OLS = sm.OLS(y, X).fit()
        max_var = max(regressor_OLS.pvalues)
        if max_var > 0.05:
            for j in range(0, num_vars - i):
                if (regressor_OLS.pvalues[j] == max_var):
                    X = np.delete(X, j, 1)
    return regressor_OLS, X

regressor_OLS, X_opt = backward_elimination(X_train, y_train)

#dataset after backward elimination
selected_features = np.where(X_opt.sum(axis=0) != 0)[0]
print(pd.DataFrame(X_opt, columns=data.columns[selected_features]))

#calculating Mean Squared Error MSE and Mean Absolute Error MAE
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, mae

      study
0  0.666667
1  0.666667
2  0.666667
3  0.333333
4  0.666667
5  0.333333
6  0.333333
7  0.000000


In [ ]:
#fitting the model
model = LinearRegression()
model.fit(X_train, y_train)

#making predictions
y_pred = model.predict(X_test)

mse, mae = calculate_metrics(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

#printing summary
print(regressor_OLS.summary())

Mean Squared Error (MSE): 9.78427116718737
Mean Absolute Error (MAE): 3.0477707006369528
                                 OLS Regression Results                                
Dep. Variable:             exam_score   R-squared (uncentered):                   0.857
Model:                            OLS   Adj. R-squared (uncentered):              0.836
Method:                 Least Squares   F-statistic:                              41.90
Date:                Wed, 20 Mar 2024   Prob (F-statistic):                    0.000343
Time:                        16:33:00   Log-Likelihood:                         -37.674
No. Observations:                   8   AIC:                                      77.35
Df Residuals:                       7   BIC:                                      77.43
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std er

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
